In [256]:
import pandas as pd

In [257]:
#loading data

data_BIS = pd.read_csv("C:/Users/Bumho/Documents/Keumkamwon/data/data_BIS.csv", encoding = "UTF-8")
data_Yeoshin = pd.read_csv("C:/Users/Bumho/Documents/Keumkamwon/data/data_Yeoshin.csv", encoding = "UTF-8")
data_ROA = pd.read_csv("C:/Users/Bumho/Documents/Keumkamwon/data/data_ROA.csv", encoding = "UTF-8")
data_self = pd.read_csv("C:/Users/Bumho/Documents/Keumkamwon/data/data_self.csv", encoding = "UTF-8")

data_BIS = data_BIS.rename(columns = {'a' : 'BIS'})
data_Yeoshin = data_Yeoshin.rename(columns = {'a' : 'Yeoshin'})
data_ROA = data_ROA.rename(columns = {'a' : 'ROA'})
data_self = data_self.rename(columns = {'a' : 'self'})

In [258]:
#merging two dataframes

record_id_BIS = []
record_id_Yeoshin = []
record_id_ROA = []
record_id_self = []

for i in range(data_BIS.shape[0]):
    record_id_BIS.append(str(data_BIS.iloc[i]['finance_cd']) + ' ' + str(data_BIS.iloc[i]['base_month']))

for i in range(data_Yeoshin.shape[0]):
    record_id_Yeoshin.append(str(data_Yeoshin.iloc[i]['finance_cd']) + ' ' + str(data_Yeoshin.iloc[i]['base_month']))

for i in range(data_ROA.shape[0]):
    record_id_ROA.append(str(data_ROA.iloc[i]['finance_cd']) + ' ' + str(data_ROA.iloc[i]['base_month']))    
    
for i in range(data_self.shape[0]):
    record_id_self.append(str(data_self.iloc[i]['finance_cd']) + ' ' + str(data_self.iloc[i]['base_month']))    
    
    
data_BIS['record_id'] = record_id_BIS
data_Yeoshin['record_id'] = record_id_Yeoshin
data_ROA['record_id'] = record_id_ROA
data_self['record_id'] = record_id_self

Merged_BIS_Yeoshin = pd.merge(data_BIS, data_Yeoshin, how="inner", left_on = "record_id", right_on = "record_id")
Merge_3 = pd.merge(Merged_BIS_Yeoshin, data_ROA, how="inner", left_on = "record_id", right_on = "record_id")
Merge_4 = pd.merge(Merge_3, data_self, how="inner", left_on = "record_id", right_on = "record_id")

Merge_4 = Merge_4[['base_month_x', 'finance_cd_x', 'finance_nm_x', 'BIS', 'Yeoshin', 'ROA', 'self']]
Merge_4 = Merge_4.ix[:,[0,2,4,6,7,8,9]]

'''
# delete March, September
check_month = []

for i in range(Merge_4.shape[0]):
    if Merge_4.iloc[i]['base_month_x'].split(" ")[0] == "3" or Merge_4.iloc[i]['base_month_x'].split(" ")[0] == "9":
        check_month.append(i)

deleted_Merge_4 = Merge_4.drop(check_month, 0)
'''
# make month, year
month=[]
year=[]
for i in range(Merge_4.shape[0]):
    month.append(int(Merge_4.iloc[i]['base_month_x'].split(" ")[0]))
    year.append(int(Merge_4.iloc[i]['base_month_x'].split(" ")[1]))
Merge_4['month'] = month
Merge_4['year'] = year



# sort by company, date

#change attribute's type to float64
sorted_Merge_4['BIS'] = sorted_Merge_4['BIS'].apply(pd.to_numeric)
sorted_Merge_4['Yeoshin'] = sorted_Merge_4['Yeoshin'].apply(pd.to_numeric)
sorted_Merge_4['ROA'] = sorted_Merge_4['ROA'].apply(pd.to_numeric)
sorted_Merge_4['self'] = sorted_Merge_4['self'].apply(pd.to_numeric)


sorted_Merge_4 = Merge_4.sort_values(['finance_cd_x', 'year', 'month'], ascending =[True, True, True])

sorted_Merge_4 = sorted_Merge_4.reset_index(drop = True)

for i in range(100000):
    if i<sorted_Merge_4.shape[0]-1:
    
        if sorted_Merge_4.iloc[i+1]['finance_cd_x'] == sorted_Merge_4.iloc[i]['finance_cd_x']:
            if sorted_Merge_4.iloc[i+1]['month'] == 12 and sorted_Merge_4.iloc[i]['month'] == 6:
                month = 9
                year = sorted_Merge_4.iloc[i]['year']
                finance_cd_x = sorted_Merge_4.iloc[i+1]['finance_cd_x']
                finance_nm_x = sorted_Merge_4.iloc[i+1]['finance_nm_x']
                base_month_x = str(month) + " " + str(year)
                BIS = 0.5 * (sorted_Merge_4.iloc[i]['BIS'] + sorted_Merge_4.iloc[i+1]['BIS'])
                Yeoshin = 0.5 * (sorted_Merge_4.iloc[i]['Yeoshin'] + sorted_Merge_4.iloc[i+1]['Yeoshin'])
                ROA = 0.5 * (sorted_Merge_4.iloc[i]['ROA'] + sorted_Merge_4.iloc[i+1]['ROA'])
                self = 0.5 * (sorted_Merge_4.iloc[i]['self'] + sorted_Merge_4.iloc[i+1]['self'])

                line = pd.DataFrame({"base_month_x" : base_month_x, "finance_cd_x" : finance_cd_x, "finance_nm_x" : finance_nm_x, "BIS" : BIS, "Yeoshin" : Yeoshin, "ROA" : ROA, "self" : self}, index=[i+1]).reset_index(drop = True)
                sorted_Merge_4 = pd.concat([sorted_Merge_4.iloc[:i+1], line, sorted_Merge_4.iloc[i+1:]])

            if sorted_Merge_4.iloc[i+1]['month'] == 6 and sorted_Merge_4.iloc[i]['month'] == 12:
                month = 3
                year = sorted_Merge_4.iloc[i+1]['year']
                finance_cd_x = sorted_Merge_4.iloc[i+1]['finance_cd_x']
                finance_nm_x = sorted_Merge_4.iloc[i+1]['finance_nm_x']
                base_month_x = str(month) + " " + str(year)
                BIS = 0.5 * (sorted_Merge_4.iloc[i]['BIS'] + sorted_Merge_4.iloc[i+1]['BIS'])
                Yeoshin = 0.5 * (sorted_Merge_4.iloc[i]['Yeoshin'] + sorted_Merge_4.iloc[i+1]['Yeoshin'])
                ROA = 0.5 * (sorted_Merge_4.iloc[i]['ROA'] + sorted_Merge_4.iloc[i+1]['ROA'])
                self = 0.5 * (sorted_Merge_4.iloc[i]['self'] + sorted_Merge_4.iloc[i+1]['self'])

                line = pd.DataFrame({"base_month_x" : base_month_x, "finance_cd_x" : finance_cd_x, "finance_nm_x" : finance_nm_x, "BIS" : BIS, "Yeoshin" : Yeoshin, "ROA" : ROA, "self" : self}, index=[i+1]).reset_index(drop = True)
                sorted_Merge_4 = pd.concat([sorted_Merge_4.iloc[:i+1], line, sorted_Merge_4.iloc[i+1:]])





# make predict variable "weak"
weak = []
for i in range(sorted_Merge_4.shape[0]-2):
    if (sorted_Merge_4.iloc[i+1]['finance_cd_x'] == sorted_Merge_4.iloc[i]['finance_cd_x']) and (sorted_Merge_4.iloc[i+2]['finance_cd_x'] == sorted_Merge_4.iloc[i]['finance_cd_x']):    
        if (float(sorted_Merge_4.iloc[i+1]['self']) < 4) or (float(sorted_Merge_4.iloc[i+2]['self'])< 4):
            weak.append(1)
        else:
            weak.append(0)
    elif sorted_Merge_4.iloc[i+1]['finance_cd_x'] == sorted_Merge_4.iloc[i]['finance_cd_x']:
        if float(sorted_Merge_4.iloc[i+1]['self']) < 4:
            weak.append(1)
        else:
            weak.append(0)
        
                                                                
    else:
        weak.append(2)

if sorted_Merge_4.iloc[sorted_Merge_4.shape[0]-1]['self']<4:
    weak.append(1)
else:
    weak.append(0)
weak.append(2)
sorted_Merge_4['weak'] = weak
final_data = sorted_Merge_4[sorted_Merge_4.weak != 2]




#final_data.to_csv("sorted_data.csv")


In [259]:
final_data

,BIS,ROA,Yeoshin,base_month_x,finance_cd_x,finance_nm_x,month,self,year,weak
0,5.100,-0.080,9.230,12 2003,10345,HK Mutual Savings Bank,12.0,3.270,2003.0,1
0,5.265,0.265,9.580,3 2004,10345,HK Mutual Savings Bank,NaN,3.365,NaN,1
1,5.430,0.610,9.930,6 2004,10345,HK Mutual Savings Bank,6.0,3.460,2004.0,1
0,5.470,0.295,11.115,9 2004.0,10345,HK Mutual Savings Bank,NaN,3.505,NaN,1
2,5.510,-0.020,12.300,12 2004,10345,HK Mutual Savings Bank,12.0,3.550,2004.0,1
0,4.910,-0.945,15.545,3 2005.0,10345,HK Mutual Savings Bank,NaN,2.930,NaN,1
3,4.310,-1.870,18.790,6 2005,10345,HK Mutual Savings Bank,6.0,2.310,2005.0,1
0,3.700,-2.045,23.025,9 2005.0,10345,HK Mutual Savings Bank,NaN,2.000,NaN,1
4,3.090,-2.220,27.260,12 2005,10345,HK Mutual Savings Bank,12.0,1.690,2005.0,1
0,3.430,-2.380,24.775,3 2006.0,10345,HK Mutual Savings Bank,NaN,1.580,NaN,1


In [279]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import cross_validation
from sklearn.metrics import precision_recall_fscore_support
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

In [288]:
#get X, Y for training

X = final_data[['ROA', 'BIS', 'Yeoshin', 'self']]
Y = final_data['weak']

# seperate training and test set
X_train, X_test, y_train, y_test= cross_validation.train_test_split(X, Y, test_size = .4, random_state = 12345)

# normalizing

sc = StandardScaler()
sc.fit(X_train)
X_train= sc.transform(X_train)
X_test= sc.transform(X_test)




In [289]:

# start logistic regression training
lr = LogisticRegression(random_state = 1234, max_iter = 50)
lr.fit(X_train, y_train)

# show scores
print("train_score : ", lr.score(X_train, y_train))
print("test_score : ", lr.score(X_test, y_test))

y_test_pred = lr.predict(X_test)
y_train_pred = lr.predict(X_train)
test_accuracy = metrics.accuracy_score(y_test, y_test_pred)
train_accuracy = metrics.accuracy_score(y_train, y_train_pred)
print(test_accuracy, train_accuracy)

print("Classification report for test set...")
print(metrics.classification_report(y_test, y_test_pred))

print("Classification report for training set...")
print(metrics.classification_report(y_train, y_train_pred))

# 행:실제 분류   열: 분류기 분류
print("Confusion matrix for test set...")
print(metrics.confusion_matrix(y_test, y_test_pred))

print("Confusion matrix for training set...")
print(metrics.confusion_matrix(y_train, y_train_pred))

train_score :  0.940860215054
test_score :  0.9444971537
0.9444971537 0.940860215054
Classification report for test set...
             precision    recall  f1-score   support

          0       0.95      0.98      0.97      1678
          1       0.92      0.80      0.85       430

avg / total       0.94      0.94      0.94      2108

Classification report for training set...
             precision    recall  f1-score   support

          0       0.95      0.98      0.96      2461
          1       0.92      0.80      0.86       701

avg / total       0.94      0.94      0.94      3162

Confusion matrix for test set...
[[1647   31]
 [  86  344]]
Confusion matrix for training set...
[[2413   48]
 [ 139  562]]


In [290]:
# random forest training

rf = RandomForestClassifier(criterion="entropy", n_estimators=100, max_depth = 4,random_state = 0  )
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [291]:
# show scores
print("train_score : ", rf.score(X_train, y_train))
print("test_score : ", rf.score(X_test, y_test))

y_test_pred = rf.predict(X_test)
y_train_pred = rf.predict(X_train)
test_accuracy = metrics.accuracy_score(y_test, y_test_pred)
train_accuracy = metrics.accuracy_score(y_train, y_train_pred)
print(test_accuracy, train_accuracy)

print("Classification report for test set...")
print(metrics.classification_report(y_test, y_test_pred))

print("Classification report for training set...")
print(metrics.classification_report(y_train, y_train_pred))

# 행:실제 분류   열: 분류기 분류
print("Confusion matrix for test set...")
print(metrics.confusion_matrix(y_test, y_test_pred))

print("Confusion matrix for training set...")
print(metrics.confusion_matrix(y_train, y_train_pred))

train_score :  0.949399114485
test_score :  0.94734345351
0.94734345351 0.949399114485
Classification report for test set...
             precision    recall  f1-score   support

          0       0.96      0.98      0.97      1678
          1       0.91      0.83      0.86       430

avg / total       0.95      0.95      0.95      2108

Classification report for training set...
             precision    recall  f1-score   support

          0       0.96      0.98      0.97      2461
          1       0.92      0.84      0.88       701

avg / total       0.95      0.95      0.95      3162

Confusion matrix for test set...
[[1642   36]
 [  75  355]]
Confusion matrix for training set...
[[2410   51]
 [ 109  592]]


In [292]:
dt = DecisionTreeClassifier(criterion="entropy", max_depth=4, random_state = 1234)
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=1234, splitter='best')

In [293]:
# show scores
print("train_score : ", dt.score(X_train, y_train))
print("test_score : ", dt.score(X_test, y_test))

y_test_pred = dt.predict(X_test)
y_train_pred = dt.predict(X_train)
test_accuracy = metrics.accuracy_score(y_test, y_test_pred)
train_accuracy = metrics.accuracy_score(y_train, y_train_pred)
print(test_accuracy, train_accuracy)

print("Classification report for test set...")
print(metrics.classification_report(y_test, y_test_pred))

print("Classification report for training set...")
print(metrics.classification_report(y_train, y_train_pred))

# 행:실제 분류   열: 분류기 분류
print("Confusion matrix for test set...")
print(metrics.confusion_matrix(y_test, y_test_pred))

print("Confusion matrix for training set...")
print(metrics.confusion_matrix(y_train, y_train_pred))

train_score :  0.950031625553
test_score :  0.943548387097
0.943548387097 0.950031625553
Classification report for test set...
             precision    recall  f1-score   support

          0       0.96      0.97      0.96      1678
          1       0.87      0.85      0.86       430

avg / total       0.94      0.94      0.94      2108

Classification report for training set...
             precision    recall  f1-score   support

          0       0.96      0.98      0.97      2461
          1       0.91      0.86      0.88       701

avg / total       0.95      0.95      0.95      3162

Confusion matrix for test set...
[[1625   53]
 [  66  364]]
Confusion matrix for training set...
[[2401   60]
 [  98  603]]


In [237]:
data_BIS

,"﻿""""",base_month,finance_cd,finance_nm,account_cd,account_nm,BIS
0,1,12 2013,12711,NH Savings Bank,SE016:G,BIS Capital Ratio,20.08
1,2,6 2014,12711,NH Savings Bank,SE016:G,BIS Capital Ratio,21.42
2,3,12 2014,12711,NH Savings Bank,SE016:G,BIS Capital Ratio,21.20
3,4,6 2015,12711,NH Savings Bank,SE016:G,BIS Capital Ratio,19.79
4,5,12 2015,12711,NH Savings Bank,SE016:G,BIS Capital Ratio,19.27
5,6,3 2016,12711,NH Savings Bank,SE016:G,BIS Capital Ratio,19.34
6,7,6 2016,12711,NH Savings Bank,SE016:G,BIS Capital Ratio,20.01
7,8,9 2016,12711,NH Savings Bank,SE016:G,BIS Capital Ratio,19.25
8,9,12 2016,12711,NH Savings Bank,SE016:G,BIS Capital Ratio,18.90
9,10,3 2017,12711,NH Savings Bank,SE016:G,BIS Capital Ratio,18.85


IndexError: single positional indexer is out-of-bounds